In [1]:
import chess
import torch
import numpy as np
from typing import Literal, Union, List, Tuple


class MatrixEncoder:
    def encode(self, board: chess.Board) -> np.ndarray:
        # 12 каналов для фигур
        board_state = np.zeros((15, 8, 8), dtype=np.float32)

        # 1. Кодируем состояние доски
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is not None:
                # Определяем канал:
                # 0-5: пешка, конь, слон, ладья, ферзь, король
                channel = piece.piece_type - 1
                if piece.color == chess.BLACK:
                    channel += 6
                row = square // 8
                col = square % 8
                board_state[channel, row, col] = 1.0

        # 2. Дополнительные признаки
        if board.has_kingside_castling_rights(chess.WHITE):
            board_state[12][7, 4] = 1.0  # Король белых на e1
        if board.has_queenside_castling_rights(chess.WHITE):
            board_state[12][7, 4] = 1.0  # Король белых на e1
        if board.has_kingside_castling_rights(chess.BLACK):
            board_state[12][7, 0] = -1.0  # Король чёрных на e8
        if board.has_queenside_castling_rights(chess.BLACK):
            board_state[12][7, 0] = -1.0  # Король чёрных на e8

        if board.ep_square is not None:
            ep_row = board.ep_square // 8
            ep_col = board.ep_square % 8
            board_state[13][ep_row, ep_col] = 1.0

        if board.peek() and board.peek().promotion is None:
            last_move = board.peek()
            if abs(last_move.from_square - last_move.to_square) == 16:  # Ход на две клетки
                double_move_row = last_move.to_square // 8
                double_move_col = last_move.to_square % 8
                board_state[14][double_move_row, double_move_col] = 1.0

        return board_state

    def get_encoded_shape(self):
        return (15, 8, 8)

In [2]:
import chess.pgn
import numpy as np
import os
from tqdm import tqdm

class PgnToNpyConverter:
    def __init__(self, pgn_path, encoder, output_dir, max_games=None, min_moves=10):
        self.pgn_path = pgn_path
        self.encoder = encoder
        self.output_dir = output_dir
        self.max_games = max_games
        self.min_moves = min_moves
        os.makedirs(output_dir, exist_ok=True)
        
    def process_games(self):
        pgn = open(self.pgn_path)
        game_count = 0
        
        with tqdm(desc="Processing games") as pbar:
            while True:
                game = chess.pgn.read_game(pgn)
                if game is None or (self.max_games is not None and game_count >= self.max_games):
                    break
                
                board = game.board()
                moves = list(game.mainline_moves())
                
                if len(moves) >= self.min_moves:
                    # Собираем все позиции партии
                    positions = []
                    for move in moves:
                        board.push(move)
                        positions.append(self.encoder.encode(board))
                    
                    # Сохраняем всю партию в один .npy файл
                    game_array = np.stack(positions)
                    np.save(os.path.join(self.output_dir, f"game_{game_count}.npy"), game_array)
                    game_count += 1
                
                pbar.update(1)
        
        print(f"Обработано {game_count} партий")

Датасет с большим количеством игр можно найти на [lichess database](https://database.lichess.org/)
Мы использовали датасет на июль 2014 года, содержащий 1,048,440 партий

In [ ]:
processor = PgnToTensorDataset(
    'C:/Users/matvey/Documents/chess_data/lichess_db_standard_rated_2014-07.pgn',
    MatrixEncoder(),
    'C:/Users/matvey/Documents/chess_data/shit/',
    max_games=10_000  # Большое количество разрозненных файлов сильно засоряет диск,
                      # поэтому если вы запускаете это локально, ставьте не больше 10_000
)
processor.process_games()